In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf 
import logging
tf.get_logger().setLevel(logging.ERROR)

EPOCHS = 32
BATCH_SIZE = 256
INPUT_FILE_NAME = '../data/frankenstein.txt'
WINDOW_LENGHT = 40
WINDOW_STEP = 3
BEAM_SIZE = 8
NUM_LETTERS = 11

2025-07-18 13:10:47.320717: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 13:10:47.321455: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 13:10:47.324625: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 13:10:47.332849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752862247.346700   19595 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752862247.35

In [ ]:
file = open(INPUT_FILE_NAME, 'r', encoding= "utf-8-sig",)
text = file.read()
file.close()

text = text.lower()
text = text.replace('\n', '')
text = text.replace('  ', ' ')

unique_charts = list(set(text))
char_to_index = dict((ch,index) for index, ch in enumerate(unique_charts))
index_to_chat = dict((index,ch) for index, ch in enumerate(unique_charts))
encoding_width = len(char_to_index)

In [24]:
fragments = []
targets = []
for i in range(0,len(text) - WINDOW_LENGHT, WINDOW_STEP):
    fragments.append(text[i: i + WINDOW_LENGHT])
    targets.append(text[i + WINDOW_LENGHT])

X = np.zeros((len(fragments),WINDOW_LENGHT,encoding_width))
y = np.zeros((len(fragments), encoding_width))

for i, fragment in enumerate(fragments):
    for j, char in enumerate(fragment):
        X[i,j, char_to_index[char]] = 1
    target_char = targets[i]
    y[i,char_to_index[target_char]] = 1


In [27]:
X[0][0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [37]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, dropout = 0.2, recurrent_dropout= 0.2, input_shape = (None, encoding_width)))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(encoding_width, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
history = model.fit(X,y, validation_split = 0.05, batch_size = BATCH_SIZE, epochs = EPOCHS, verbose = 2, shuffle = True)

Epoch 1/32
533/533 - 148s - 278ms/step - loss: 2.8197 - val_loss: 2.7427
Epoch 2/32
533/533 - 131s - 246ms/step - loss: 2.4520 - val_loss: 2.6121
Epoch 3/32
533/533 - 123s - 230ms/step - loss: 2.3322 - val_loss: 2.5430
Epoch 4/32
533/533 - 119s - 223ms/step - loss: 2.2494 - val_loss: 2.4853
Epoch 5/32
533/533 - 120s - 225ms/step - loss: 2.1859 - val_loss: 2.4496
Epoch 6/32
533/533 - 118s - 221ms/step - loss: 2.1360 - val_loss: 2.4180
Epoch 7/32
533/533 - 117s - 220ms/step - loss: 2.0939 - val_loss: 2.3864
Epoch 8/32
533/533 - 118s - 222ms/step - loss: 2.0591 - val_loss: 2.3683
Epoch 9/32
533/533 - 119s - 223ms/step - loss: 2.0256 - val_loss: 2.3489
Epoch 10/32
533/533 - 120s - 224ms/step - loss: 1.9966 - val_loss: 2.3348
Epoch 11/32
533/533 - 120s - 225ms/step - loss: 1.9760 - val_loss: 2.3234
Epoch 12/32
533/533 - 118s - 222ms/step - loss: 1.9524 - val_loss: 2.3078
Epoch 13/32
533/533 - 117s - 220ms/step - loss: 1.9325 - val_loss: 2.3057
Epoch 14/32
533/533 - 119s - 223ms/step - loss:

In [ ]:
letters = "the body "
one_hots = []
for i, char in enumerate(letters):
    x = np.zeros(encoding_width)
    x[char_to_index[char]] = 1
    one_hots.append(x)
beams = [(np.log(1.0),letters, one_hots)]

for i in range(70):
    minibatch_list = []

    for triple in beams: 
        minibatch_list.append(triple[2])
    minibatch = np.array(minibatch_list)
    y_predict = model.predict(minibatch, verbose = 0)
    new_beams = []
    for j , softmax_vec in enumerate(y_predict):
        triple = beams[j]
        for k in range(BEAM_SIZE):
            char_index = np.argmax(softmax_vec)
            new_prob = triple[0] + np.log(softmax_vec[char_index])
            new_letters = triple[1] + index_to_chat[char_index]
            x = np.zeros(encoding_width)
            x[char_index] = 1
            new_one_hots = triple[2].copy()
            new_one_hots.append(x)
            new_beams.append((new_prob,new_letters,new_one_hots))
            softmax_vec[char_index] = 0
    new_beams.sort(key=lambda tup: tup[0], reverse= True)
    beams = new_beams[0:BEAM_SIZE]
for item in beams:
    print(item[1])
            

the body which i had been that i had been the father, and the father, and the s
the body which i had been the father, and the father, and the father, and the s
the body which i had been that i had been the father, and the father, and the f
the body which i had been that i had been the father, and the father, and the m
the body which i had been the father, and the father, and the father, and the f
the body which i had been that i had been the father, and the father of the cou
the body which i had been the father, and the father, and the father, and the m
the body which i had been that i had been the father, and the father, and the c


In [ ]:
import numpy as np
import copy

letters = "the body "
one_hots = []
for char in letters:
    x = np.zeros(encoding_width)
    x[char_to_index[char]] = 1
    one_hots.append(x)

beams = [(np.log(1.0), letters, one_hots)]

for _ in range(NUM_LETTERS):
    minibatch_list = [beam[2] for beam in beams]
    minibatch = np.array(minibatch_list)  # shape: (beam_size, seq_len, encoding_width)
    
    y_predict = model.predict(minibatch, verbose=0)
    new_beams = []

    for j, softmax_vec in enumerate(y_predict):
        log_prob, sequence, one_hot_seq = beams[j]
        top_k_indices = np.argsort(softmax_vec)[-BEAM_SIZE:][::-1]

        for idx in top_k_indices:
            prob = softmax_vec[idx]
            if prob == 0:  # Avoid log(0)
                continue
            new_log_prob = log_prob + np.log(prob)
            new_sequence = sequence + index_to_char[idx]
            x = np.zeros(encoding_width)
            x[idx] = 1
            new_one_hot_seq = [vec.copy() for vec in one_hot_seq]
            new_one_hot_seq.append(x)
            new_beams.append((new_log_prob, new_sequence, new_one_hot_seq))

    new_beams.sort(key=lambda tup: tup[0], reverse=True)
    beams = new_beams[:BEAM_SIZE]

for log_prob, text, _ in beams:
    print(text)

the body ooooooooooo
the body *oooooooooo
the body ]oooooooooo
the body uoooooooooo
the body 9oooooooooo
the body o*ooooooooo
the body yoooooooooo
the body 5oooooooooo


In [ ]:
import numpy as np

sequence = "the body"
one_hots = []
for char in sequence:
    x = np.zeros(encoding_width)
    x[char_to_index[char]] = 1
    one_hots.append(x)

input_sequence = np.array([one_hots]) 
y_pred = model.predict(input_sequence, verbose=0)[0]  

predicted_index = np.argmax(y_pred)
predicted_char = index_to_char[predicted_index]

print("Siguiente letra predicha:", predicted_char)